In [13]:
from jinja2 import Environment, FileSystemLoader, select_autoescape
import json
from skimage import io
import numpy as np
from os import listdir
from fpdf import FPDF
import math
import glob
import os
import asyncio
from pathlib import Path
from pyppeteer import launch
import nest_asyncio
nest_asyncio.apply()

In [34]:
path = '/home/edoelas/git/card-generator'
template = "templates/front.html"
deck = "decks/basic.json"


async def capture_screenshot(url, png_path):
    browser = await launch()
    page = await browser.newPage()
    await page.setViewport({'width': 1000, 'height': 2000})
    await page.goto(url)
    await page.screenshot({'path': Path(png_path)})
    await browser.close()

def crop_card(img):
    white = np.array([255, 255, 255, 255])
    mask = np.abs(img - white).sum(axis=2) < 0.05

    # Find the bounding box of those pixels
    coords = np.array(np.nonzero(~mask))
    top_left = np.min(coords, axis=1)
    bottom_right = np.max(coords, axis=1)

    margin = 1
    out = img[top_left[0]:bottom_right[0],
                top_left[1]:bottom_right[1]]
    new_shape = tuple(map(sum, zip(out.shape,(margin*2,margin*2,0))))
    new_image = np.full(new_shape,255,dtype=np.uint8)
    new_image[margin:-margin,margin:-margin] = out
    return new_image

template = env.get_template(template)
data = json.load(open(deck))

env = Environment(
    loader=FileSystemLoader("./"),
    autoescape=select_autoescape()
)

# Delete old files
for f in  glob.glob('./png_out/*'): os.remove(f)
for f in  glob.glob('./html_out/*'): os.remove(f)
    
# Generate new images
for card in data:
    html_path = './html_out/'+card['name']+'.html'
    png_path = './png_out/'+card['name']+'.png'
    url = 'file://{}/html_out/{}.html'.format(path, card['name'])

    template.stream(card).dump(html_path) # Create HTML

    asyncio.get_event_loop().run_until_complete(capture_screenshot(url, png_path))
    image = io.imread(png_path)
    image = crop_card(image) # Crop image

    for i in range(card['amount']): # Multiply number of images
        png_path_copies = './png_out/{}_{}.png'.format(card['name'],i+1)
        io.imsave(png_path_copies,image)
    os.remove(png_path)

# Generate PDF
cards_path_list = listdir('./png_out')
cards_path_list.sort()
num_pages = math.ceil(len(cards_path_list)/9)
width = 2.5
height = 3.5
margin = 0.4
pdf = FPDF('P', 'in', 'A4')

for index, card_path in enumerate(cards_path_list):
    if index % 9 == 0: pdf.add_page()
    pos = index % 9
    x_pos = margin + width*(pos%3)
    y_pos = margin + height*math.floor(pos/3)
    pdf.image('png_out/'+cards_path_list[index], x = x_pos, y = y_pos, w=2.5)

pdf.output('./pdf_out/test.pdf', 'F')

''